In [1]:
# Create the directory if it doesn't exist
!mkdir -p ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

La sintaxis del comando no es correcta.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 16 1330k   16  222k    0     0   289k      0  0:00:04 --:--:--  0:00:04  292k
100 1330k  100 1330k    0     0  1438k      0 --:--:-- --:--:-- --:--:-- 1450k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1253k  100 1253k    0     0  2998k      0 --:--:-- --:--:-- --:--:-- 3057k


In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
import dagshub
import mlflow

dagshub.init(repo_owner='colome8', repo_name='nyc-taxi-time-prediction', mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()
print("MLFLOW_TRACKING_URI:", MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as colome8

Initialized MLflow to track repo "colome8/nyc-taxi-time-prediction"

Repository colome8/nyc-taxi-time-prediction initialized!

MLFLOW_TRACKING_URI: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/52f4281e23894b08be828e86a3d02bc9', creation_time=1726632055317, experiment_id='0', last_update_time=1726632055317, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [10]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("colome8/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [11]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [13]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:8.01593                           
[1]	validation-rmse:7.20039                           
[2]	validation-rmse:6.60631                           
[3]	validation-rmse:6.17529                           
[4]	validation-rmse:5.87501                           
[5]	validation-rmse:5.66721                           
[6]	validation-rmse:5.52240                           
[7]	validation-rmse:5.42373                           
[8]	validation-rmse:5.35442                           
[9]	validation-rmse:5.30810                           
[10]	validation-rmse:5.27784                          
[11]	validation-rmse:5.25299                          
[12]	validation-rmse:5.23475                          
[13]	validation-rmse:5.22309                          
[14]	validation-rmse:5.21325                          
[15]	validation-rmse:5.20743                          
[16]	validation-rmse:5.20302                          
[17]	validation-rmse:5.19886                          
[18]	valid

2024/11/12 20:55:17 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/12 20:55:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciadatos\nyc-taxi-time-prediction\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:55:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/12 20:55:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciad

[0]	validation-rmse:7.40958                                                     
[1]	validation-rmse:6.40934                                                     
[2]	validation-rmse:5.85205                                                     
[3]	validation-rmse:5.55717                                                     
[4]	validation-rmse:5.40267                                                     
[5]	validation-rmse:5.32324                                                     
[6]	validation-rmse:5.28420                                                     
[7]	validation-rmse:5.25907                                                     
[8]	validation-rmse:5.24386                                                     
[9]	validation-rmse:5.23404                                                     
[10]	validation-rmse:5.22718                                                    
[11]	validation-rmse:5.22290                                                    
[12]	validation-rmse:5.22053

2024/11/12 20:57:25 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/12 20:57:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciadatos\nyc-taxi-time-prediction\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:57:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/12 20:57:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciad

[0]	validation-rmse:6.41147                                                     
[1]	validation-rmse:5.56672                                                     
[2]	validation-rmse:5.33690                                                     
[3]	validation-rmse:5.27639                                                     
[4]	validation-rmse:5.25848                                                     
[5]	validation-rmse:5.24271                                                     
[6]	validation-rmse:5.23716                                                     
[7]	validation-rmse:5.23492                                                     
[8]	validation-rmse:5.23648                                                     
[9]	validation-rmse:5.23226                                                     
[10]	validation-rmse:5.23187                                                    
[11]	validation-rmse:5.23374                                                    
[12]	validation-rmse:5.23329

...

2024/11/12 21:11:21 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/12 21:11:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciadatos\nyc-taxi-time-prediction\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:11:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/12 21:11:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciad

[0]	validation-rmse:5.75983                                                     
[1]	validation-rmse:5.42060                                                     
[2]	validation-rmse:5.36116                                                     
[3]	validation-rmse:5.34686                                                     
[4]	validation-rmse:5.33198                                                     
[5]	validation-rmse:5.31459                                                     
[6]	validation-rmse:5.30745                                                     
[7]	validation-rmse:5.30058                                                     
[8]	validation-rmse:5.29659                                                     
[9]	validation-rmse:5.29382                                                     
[10]	validation-rmse:5.29218                                                    
[11]	validation-rmse:5.28830                                                    
[12]	validation-rmse:5.28581

2024/11/12 21:12:32 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/12 21:12:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciadatos\nyc-taxi-time-prediction\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:12:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/12 21:12:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciad

100%|██████████| 10/10 [19:43<00:00, 118.36s/trial, best loss: 5.182970213294785]
[0]	validation-rmse:8.01593
[1]	validation-rmse:7.20039
[2]	validation-rmse:6.60631
[3]	validation-rmse:6.17529
[4]	validation-rmse:5.87501
[5]	validation-rmse:5.66721
[6]	validation-rmse:5.52240
[7]	validation-rmse:5.42373
[8]	validation-rmse:5.35442
[9]	validation-rmse:5.30810
[10]	validation-rmse:5.27784
[11]	validation-rmse:5.25299
[12]	validation-rmse:5.23475
[13]	validation-rmse:5.22309
[14]	validation-rmse:5.21325
[15]	validation-rmse:5.20743
[16]	validation-rmse:5.20302
[17]	validation-rmse:5.19886
[18]	validation-rmse:5.19659
[19]	validation-rmse:5.19286
[20]	validation-rmse:5.18887
[21]	validation-rmse:5.18699
[22]	validation-rmse:5.18627
[23]	validation-rmse:5.18537
[24]	validation-rmse:5.18480
[25]	validation-rmse:5.18421
[26]	validation-rmse:5.18438
[27]	validation-rmse:5.18182
[28]	validation-rmse:5.18238
[29]	validation-rmse:5.18282
[30]	validation-rmse:5.18305
[31]	validation-rmse:5.18358


2024/11/12 21:14:41 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/11/12 21:14:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciadatos\nyc-taxi-time-prediction\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:14:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/11/12 21:14:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/12 21:14:58 INFO mlflow.tracking._tracking_service.cli

In [15]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Successfully registered model 'nyc-taxi-model'.
2024/11/12 21:16:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 1
Created version '1' of model 'nyc-taxi-model'.


In [16]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1731467793073, current_stage='None', description='The model version 1 was transitioned to champion on 2024-11-12 21:16:33.281430', last_updated_timestamp=1731467793602, name='nyc-taxi-model', run_id='390e951af5e541b0a795144f1ce93032', run_link='', source='mlflow-artifacts:/52f4281e23894b08be828e86a3d02bc9/390e951af5e541b0a795144f1ce93032/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [17]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([19.171764, 28.428782,  9.392993, ..., 46.923378, 14.162955,
       19.9719  ], dtype=float32)